In [1]:
# library
import tensorflow as tf
import keras
import keras.layers as kl
from keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.layers.core import Dropout, Reshape, Dense, Activation, Flatten
from keras.layers import BatchNormalization, InputLayer, Input
from keras import models
from keras.models import Sequential, Model
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from keras.callbacks import EarlyStopping, History, ModelCheckpoint
import IOHelper
import SequenceHelper
import pandas as pd
import numpy as np
import sys
sys.path.append('Neural_Network_DNA_Demo/')
import random
random.seed(1234)
import torch.nn as nn
import torch.nn.functional as F
from eugene.models.base import BaseModel
# Load in relevant libraries, and alias where appropriate
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.optim as optim
from torchvision import datasets, transforms 
from torch.utils import data
import seaborn as sns
import math as math
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.io as pio
import os
import plotly.graph_objects as go
from scipy.stats import spearmanr
from scipy import stats
from sklearn.metrics import mean_squared_error
import plotly.figure_factory as ff
import upsetplot
import umap
from Bio import SeqIO
from Levenshtein import distance
from matplotlib_venn import venn3
import re
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold, cross_val_score
pd.set_option('max_colwidth',260)
from sklearn.model_selection import train_test_split
from sklearn import metrics
import itertools
from itertools import combinations
from scipy.stats import ttest_ind
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
from scipy.stats import pearsonr
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import GradientBoostingRegressor
import plotly.subplots as sp
from sklearn.metrics import r2_score
import statsmodels.api as sm
from statsmodels.regression.linear_model import OLS
from sklearn.model_selection import cross_val_predict
from kerastuner import HyperModel
from kerastuner.tuners import RandomSearch, BayesianOptimization, Hyperband

Global seed set to 13
<ipython-input-1-1f1e2eabb756>:73: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner import HyperModel


In [2]:
# function to load sequences and enhancer activity
def prepare_input(set):
    # Convert sequences to one-hot encoding matrix
    file_seq = str("Seq_" + set + ".fa")
    input_fasta_data_A = IOHelper.get_fastas_from_file(file_seq, uppercase=True)

    # get length of first sequence
    sequence_length = len(input_fasta_data_A.sequence.iloc[0])

    # Convert sequence to one hot encoding matrix
    seq_matrix_A = SequenceHelper.do_one_hot_encoding(input_fasta_data_A.sequence, sequence_length,
                                                      SequenceHelper.parse_alpha_to_seq)
    print(seq_matrix_A.shape)
    
    X = np.nan_to_num(seq_matrix_A) # Replace NaN with zero and infinity with large finite numbers
    X_reshaped = X.reshape((X.shape[0], X.shape[1], X.shape[2]))

    Activity = pd.read_table("Seq_activity_" + set + ".txt")
    Y_dev = Activity.Dev_log2_enrichment
    Y_hk = Activity.Hk_log2_enrichment
    Y = [Y_dev, Y_hk]
    
    print(set)

    return input_fasta_data_A.sequence, seq_matrix_A, X_reshaped, Y

# Data for train/val/test sets
X_train_sequence, X_train_seq_matrix, X_train, Y_train = prepare_input("Train")
X_valid_sequence, X_valid_seq_matrix, X_valid, Y_valid = prepare_input("Val")
X_test_sequence, X_test_seq_matrix, X_test, Y_test = prepare_input("Test")

(3525, 248, 4)
Train
(1512, 248, 4)
Val
(482, 248, 4)
Test


In [3]:
### Additional metrics
from scipy.stats import spearmanr
def Spearman(y_true, y_pred):
     return ( tf.py_function(spearmanr, [tf.cast(y_pred, tf.float32), 
                       tf.cast(y_true, tf.float32)], Tout = tf.float32) )

In [4]:
# model
params = {'batch_size': 128,
          'epochs': 100,
          'early_stop': 10,
          'kernel_size1': 7,
          'kernel_size2': 3,
          'kernel_size3': 5,
          'kernel_size4': 3,
          'learning_rate': 0.001,
          'num_filters': 256,
          'num_filters2': 60,
          'num_filters3': 60,
          'num_filters4': 120,
          'n_conv_layer': 4,
          'n_add_layer': 2,
          'dropout_prob': 0.4,
          'dense_neurons1': 256,
          'dense_neurons2': 256,
          'pad':'same'}

### Additional metrics
from scipy.stats import spearmanr
def Spearman(y_true, y_pred):
     return ( tf.py_function(spearmanr, [tf.cast(y_pred, tf.float32), 
                       tf.cast(y_true, tf.float32)], Tout = tf.float32) )
def SynIgP(params=params):
    
    learning_rate = params['learning_rate']
    dropout_prob = params['dropout_prob']
    n_conv_layer = params['n_conv_layer']
    n_add_layer = params['n_add_layer']
    
    # body
    input = kl.Input(shape=(248, 4))
    x = kl.Conv1D(params['num_filters'], kernel_size=params['kernel_size1'],
                  padding=params['pad'],
                  name='Conv1D_1st')(input)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling1D(2)(x)

    for i in range(1, n_conv_layer):
        x = kl.Conv1D(params['num_filters'+str(i+1)],
                      kernel_size=params['kernel_size'+str(i+1)],
                      padding=params['pad'],
                      name=str('Conv1D_'+str(i+1)))(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
        x = MaxPooling1D(2)(x)
    
    x = Flatten()(x)
    
    # dense layers
    for i in range(0, n_add_layer):
        x = kl.Dense(params['dense_neurons'+str(i+1)],
                     name=str('Dense_'+str(i+1)))(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
        x = Dropout(dropout_prob)(x)
    bottleneck = x
    
    # heads per task (rep1 and rep2)
    tasks = ['A', 'B']
    outputs = []
    for task in tasks:
        outputs.append(kl.Dense(1, activation='linear', name=str('Dense_' + task))(bottleneck))

    model = keras.models.Model([input], outputs)
    model.compile(keras.optimizers.Adam(learning_rate=learning_rate),
                  loss=['mse', 'mse'], # loss
                  loss_weights=[1, 1], # loss weigths to balance
                  metrics=[Spearman]) # additional track metric

    return model, params

SynIgP()[0].summary()
SynIgP()[1] # dictionary

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 248, 4)]     0           []                               
                                                                                                  
 Conv1D_1st (Conv1D)            (None, 248, 256)     7424        ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 248, 256)    1024        ['Conv1D_1st[0][0]']             
 alization)                                                                                       
                                                                                                  
 activation (Activation)        (None, 248, 256)     0           ['batch_normalization[0][0]']

{'batch_size': 128,
 'epochs': 100,
 'early_stop': 10,
 'kernel_size1': 7,
 'kernel_size2': 3,
 'kernel_size3': 5,
 'kernel_size4': 3,
 'learning_rate': 0.001,
 'num_filters': 256,
 'num_filters2': 60,
 'num_filters3': 60,
 'num_filters4': 120,
 'n_conv_layer': 4,
 'n_add_layer': 2,
 'dropout_prob': 0.4,
 'dense_neurons1': 256,
 'dense_neurons2': 256,
 'pad': 'same'}

In [5]:
#Training 

def train(selected_model, X_train, Y_train, X_valid, Y_valid, params):

    my_history=selected_model.fit(X_train, Y_train,
                                  validation_data=(X_valid, Y_valid),
                                  batch_size=params['batch_size'], epochs=params['epochs'],
                                  callbacks=[EarlyStopping(patience=params['early_stop'], monitor="val_loss", restore_best_weights=True),
                                             History()])
    
    return selected_model, my_history


main_model, main_params = SynIgP()
main_model, my_history = train(main_model, X_train, Y_train, X_valid, Y_valid, main_params)


Epoch 1/100
28/28 [==============================] - 15s 445ms/step - loss: 7.9561 - Dense_A_loss: 3.7106 - Dense_B_loss: 4.2454 - Dense_A_Spearman: 0.4603 - Dense_B_Spearman: 0.4396 - val_loss: 9.9538 - val_Dense_A_loss: 4.8922 - val_Dense_B_loss: 5.0616 - val_Dense_A_Spearman: 0.5136 - val_Dense_B_Spearman: 0.5129
Epoch 2/100
28/28 [==============================] - 12s 427ms/step - loss: 5.6483 - Dense_A_loss: 2.7271 - Dense_B_loss: 2.9212 - Dense_A_Spearman: 0.5855 - Dense_B_Spearman: 0.5665 - val_loss: 9.9125 - val_Dense_A_loss: 4.5813 - val_Dense_B_loss: 5.3313 - val_Dense_A_Spearman: 0.5359 - val_Dense_B_Spearman: 0.5380
Epoch 3/100
28/28 [==============================] - 12s 432ms/step - loss: 5.1885 - Dense_A_loss: 2.5468 - Dense_B_loss: 2.6418 - Dense_A_Spearman: 0.6206 - Dense_B_Spearman: 0.6118 - val_loss: 9.3640 - val_Dense_A_loss: 4.2560 - val_Dense_B_loss: 5.1080 - val_Dense_A_Spearman: 0.5232 - val_Dense_B_Spearman: 0.5023
Epoch 4/100
28/28 [===========================

In [6]:
#Evaluating the Model

from scipy import stats
from sklearn.metrics import mean_squared_error

# create functions
def summary_statistics(X, Y, set, task):
    pred = main_model.predict(X, batch_size=main_params['batch_size'])
    if task =="A":
        i=0
    if task =="B":
        i=1
    print(set + ' MSE ' + task + ' = ' + str("{0:0.2f}".format(mean_squared_error(Y, pred[i].squeeze()))))
    print(set + ' PCC ' + task + ' = ' + str("{0:0.2f}".format(stats.pearsonr(Y, pred[i].squeeze())[0])))
    print(set + ' SCC ' + task + ' = ' + str("{0:0.2f}".format(stats.spearmanr(Y, pred[i].squeeze())[0])))
    
# run for each set and enhancer type
summary_statistics(X_train, Y_train[0], "train", "A")
summary_statistics(X_valid, Y_valid[0], "validation", "A")
summary_statistics(X_test, Y_test[0], "test", "A")


28/28 [==============================] - 2s 66ms/step
train MSE A = 0.23
train PCC A = 0.98
train SCC A = 0.98
12/12 [==============================] - 1s 65ms/step
validation MSE A = 2.78
validation PCC A = 0.56
validation SCC A = 0.58
4/4 [==============================] - 0s 64ms/step
test MSE A = 2.01
test PCC A = 0.65
test SCC A = 0.66


In [7]:
# save model
#Save model weights

model_name="SynIgP"

model_json = main_model.to_json()
with open('Model_' + model_name + '.json', "w") as json_file:
    json_file.write(model_json)
main_model.save_weights('Model_' + model_name + '.h5')

#save entire model
main_model.save('saved_model/my_model')

In [8]:
# model predict

seq = # 248 bp DNA sequences
seq = np.array(seq)
seq_matrix = SequenceHelper.do_one_hot_encoding(seq, 248, SequenceHelper.parse_alpha_to_seq)

model_ID = 'SynIgP'

### load model
def load_model(model_path):
    import deeplift
    from keras.models import model_from_json
    keras_model_weights = model_path + '.h5'
    keras_model_json = model_path + '.json'
    keras_model = model_from_json(open(keras_model_json).read())
    keras_model.load_weights(keras_model_weights)
    #keras_model.summary()
    return keras_model, keras_model_weights, keras_model_json

keras_model, keras_model_weights, keras_model_json = load_model(model_ID)


pred=keras_model.predict(seq_matrix)




SyntaxError: invalid syntax (<ipython-input-8-aa151cbd3d32>, line 3)